In [ ]:
import pandas as pd

In [ ]:
pdf_data  = pd.read_csv('/content/output.csv',index_col=0)
pdf_data

In [ ]:
!pip install yake

In [ ]:
import yake
# import lower

In [ ]:
language = "en"
max_ngram_size = 3
deduplication_thresold = 0.4
deduplication_algo = 'leve'
windowSize = 2
numOfKeywords = 20

extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)

keywords = []
lower_keywords = []

for index,row in pdf_data.iterrows():
    text = row['pagedata']
    kws = extractor.extract_keywords(text)
    for kw in kws:
        keywords.append(kw[0])
        lower_keywords.append(kw[0].lower())
    

print(list(set(keywords)))
print(list(set(lower_keywords)))
# lower_keywords =  lower(keywords)

['Banking Partners', 'Organization Size Outlook', 'Submit', 'info', 'North America', 'Supply Forecasting', 'Supply Chain Construction', 'Market Overview', 'Domino Data Lab', 'Enterprise Resource Planning', 'United States', 'FREE Sample Copy', 'Future', 'INR', 'detailed document', 'Predictive Analytics', 'Overview', 'highest market share', 'Order Payment Agreement', 'larger market share', 'Market share analysis', '6:12', '2019-2026', '2021-2028 product development', 'Email', 'future requirements', 'Segment Forecasts', 'Advanced', 'Sales and Marketing', 'future', 'Report Overview', 'industry-analysis', 'Outlook', 'CONNECT', 'customers and increase', 'Factors', 'Number of Pages', 'analytics market growth', 'Share', 'organization size', 'York City', 'Competitors Pricing Strategy', 'growth', 'Contents Research Methodology', 'Global Advanced', 'Communications Technology Data', 'CAGR', 'Revenue', 'solutions', '12:54', 'Data Analytics', 'Predictive Analytics Market', 'level', 'Job Title', 'Nor

In [ ]:
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')

final_synonyms = []
synonyms = []
antonyms = []

for word in lower_keywords:
    word_syn = {}
    syns = []
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            # print('word : ',word)
            # print('syn',l.name())
            # print('\n\n')
            synonyms.append(l.name())
            syns.append(l.name())
            word_syn['keyword'] = word
            word_syn['synonyms'] = list(set(syns))
            final_synonyms.append(word_syn)
            if l.antonyms():
                antonyms.append(l.antonyms()[0].name())
  
# print(set(synonyms))
print(final_synonyms[:4])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[{'keyword': 'predictive', 'synonyms': ['prognostic', 'predictive', 'prognosticative']}, {'keyword': 'predictive', 'synonyms': ['prognostic', 'predictive', 'prognosticative']}, {'keyword': 'predictive', 'synonyms': ['prognostic', 'predictive', 'prognosticative']}, {'keyword': 'size', 'synonyms': ['sizing', 'size_of_it', 'size']}]


In [ ]:
synonyms_df = pd.DataFrame(final_synonyms)
synonyms_df = synonyms_df.drop_duplicates(subset=['keyword']).reset_index(drop=True)
synonyms_df

,keyword,synonyms
0,predictive,"[prognostic, predictive, prognosticative]"
1,size,"[sizing, size_of_it, size]"
2,supply,"[furnish, cater, supply, supplying, issue, add..."
3,data,"[data_point, datum, data, information]"
4,market,"[securities_industry, marketplace, commerciali..."
...,...,...
78,testimonials,"[recommendation, testimonial, tribute, good_wo..."
79,client,"[guest, client, customer, node]"
80,view,"[consider, sentiment, regard, scene, position,..."
81,grand,"[terrific, idealistic, rarefied, lordly, 1000,..."


In [ ]:
synonyms_df.to_csv('synonyms_df.csv',index=False)

In [ ]:
result_dict= []

for index,row in pdf_data.iterrows():
    
    text = row['pagedata']
    id = row['id']
    text_list = text.split('\n')
    for each in text_list:
        #print(each)
        #res = any(ele in each for ele in keywords)
        res = [item for item in keywords if(item in each)]
        
        #print(res)
        if bool(res) == True:
            result = {}
            #print(each)
            print(res)
            print(each)
            result['id'] = id
            result['text_sentence'] = each
            result['keywords'] = list(set(res))
            #result['text_sentence'] = each

        result_dict.append(result)        


print(' *****      RESLUT  ******** \n\n')
print(result_dict)


['Predictive Analytics Market', 'Market Size', 'Trends Report', 'Analytics', 'Predictive', 'Size', '2019-2026', 'Predictive Analytics Market', 'Market', 'Predictive', 'Analytics', 'Trends Report', '6:12', 'Size', 'Analytics Market Size', 'Trends Report', 'Predictive Analytics', 'Size', 'Market', 'Predictive Analytics Market', 'Market', 'Analytics', 'Report', 'Size', 'Predictive Analytics Market', 'Trends Report', '6:12 PM Predictive', 'Analytics', 'Size', 'Market Size', '6:12', 'Trends Report', 'Analytics Market Size', 'Size', '2019-2026', '6:12', 'Predictive Analytics', 'Market', '6:12 PM Predictive', 'Analytics Market Size', 'Trends Report', 'Predictive Analytics', '6:12 PM Predictive', '2019-2026', 'Size', 'Market', '6:12', 'Analytics Market Size', 'Trends Report', 'Size', '2019-2026', 'Market', 'Predictive Analytics', '6:12', '6:12 PM Predictive', 'Trends', 'Market', 'Predictive Analytics', 'Size', 'Analytics Market Size', 'Trends', 'Market', 'Size', 'Market', 'Size', 'Market Size'

In [ ]:
keyword_df = pd.DataFrame(result_dict)

In [ ]:
keyword_df.to_csv('keywords_from_pdf.csv',index=False)

## BERT BASED SYNONYMS 

 -- Referrecned from https://gist.github.com/Amirosimani/1c055d73e7018ea1eb5b4576d4d327ea

In [ ]:
!pip install bert-embedding mxnet-cu100

     |████████████████████████████████| 352.6 MB 11 kB/s 
     |████████████████████████████████| 13.8 MB 9.8 MB/s 
     |████████████████████████████████| 29.6 MB 1.5 MB/s 
     |████████████████████████████████| 209 kB 59.6 MB/s 
     |████████████████████████████████| 344.4 MB 2.0 kB/s 
     |████████████████████████████████| 827.8 MB 8.8 kB/s 
     |████████████████████████████████| 540.1 MB 28 kB/s 
     |████████████████████████████████| 444.8 MB 6.0 kB/s 
     |████████████████████████████████| 540.1 MB 28 kB/s 
     |████████████████████████████████| 488.3 MB 14 kB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.6.0-py3-none-any.whl size=259930 sha256=1e10c075ccd72982ae52750d752e69f52d2d40580d2e01649dd75de79e230823
  Stored in directory: /root/.cache/pip/wheels/a6/41/8f/45bd1c58055d87aee5a71b6756a427ea8d92e506b3a9d17370
Successfully built gluonnlp
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully u

In [ ]:
import mxnet as mx
from bert_embedding import BertEmbedding

In [ ]:
ctx = mx.gpu(0)
bert = BertEmbedding(ctx=ctx)

Vocab file is not found. Downloading.


MXNetError: ignored

## With HuggingFace

In [ ]:
!pip install transformers
!pip install --upgrade jax jaxlib
!pip install --upgrade git+https://github.com/google/flax.git


     |████████████████████████████████| 3.1 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 67.0 MB/s 
     |████████████████████████████████| 3.3 MB 43.8 MB/s 
     |████████████████████████████████| 895 kB 55.1 MB/s 
     |████████████████████████████████| 61 kB 382 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 61.5 MB 80 kB/s 
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.1.71+cuda111
    Uninstalling jaxlib-0.1.71+cuda111:
      Successfully uninstalled jaxlib-0.1.71+cuda111
  Cloning https://github.com/google/flax.git to /tmp/pip-req-build-_y0bgybk
  Running command git clone -q https://github.com/google/flax.git /tmp/pip-req-build-_y0bgybk
     |████████████████████████████████| 126 kB 4.2 MB/s 
     |████████████████████████████████| 65 kB 2.9 MB/s 
  Created wheel for flax: filename=flax-0.3.6

In [ ]:
!wget http://pcai056.informatik.uni-leipzig.de/downloads/corpora/eng-com_web-public_2018_10K.tar.gz
!tar -xzvf eng-com_web-public_2018_10K.tar.gz

--2021-12-02 12:39:10--  http://pcai056.informatik.uni-leipzig.de/downloads/corpora/eng-com_web-public_2018_10K.tar.gz
Resolving pcai056.informatik.uni-leipzig.de (pcai056.informatik.uni-leipzig.de)... 139.18.2.216
Connecting to pcai056.informatik.uni-leipzig.de (pcai056.informatik.uni-leipzig.de)|139.18.2.216|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2310663 (2.2M) [application/x-gzip]
Saving to: ‘eng-com_web-public_2018_10K.tar.gz’

eng-com_web-public_ 100%[===================>]   2.20M  1.17MB/s    in 1.9s    

2021-12-02 12:39:12 (1.17 MB/s) - ‘eng-com_web-public_2018_10K.tar.gz’ saved [2310663/2310663]

eng-com_web-public_2018_10K/
eng-com_web-public_2018_10K/eng-com_web-public_2018_10K-sentences.txt
eng-com_web-public_2018_10K/eng-com_web-public_2018_10K-inv_so.txt
eng-com_web-public_2018_10K/eng-com_web-public_2018_10K-import.sql
eng-com_web-public_2018_10K/eng-com_web-public_2018_10K-co_s.txt
eng-com_web-public_2018_10K/eng-com_web-public_2018_10K

In [ ]:
with open('eng-com_web-public_2018_10K/eng-com_web-public_2018_10K-sentences.txt', 'r') as f:
    lines = f.readlines()

In [ ]:
lines[0]

'1\t"100 Butches Number 12" moved to the West and was disconcerted to find that she didn\'t draw lesbian attention in the way she had in Singapore.\n'

In [ ]:
all_sentences = [l.split('\t')[1] for l in lines] 

In [ ]:
len(all_sentences)

10000

In [ ]:
from transformers import AutoTokenizer, BertModel
import torch

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

sentance = ["Hello, my dog is cute","Hello, my dog is super cute and the cat i have is jealous of her"]

inputs = tokenizer(sentance, return_tensors="pt",padding=True,truncation=True)
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
!nvideia-smi

/bin/bash: nvideia-smi: command not found


In [ ]:
word_vect = outputs.pooler_output.detach().numpy()

In [ ]:
tokenizer.convert_ids_to_tokens(last_hidden_states[0])

ValueError: ignored

In [ ]:
!nvidia-smi

Tue Dec  7 05:28:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-beeb5a54-7124-9ddf-8464-829c26539b08)


In [ ]:
inputs.input_ids.tolist()

[[101, 7592, 1010, 2026, 3899, 2003, 10140, 102, 0],
 [101, 7592, 1010, 2026, 3899, 2003, 3565, 10140, 102]]

In [ ]:
from sklearn.neighbors import KDTree
import numpy as np
from transformers import AutoTokenizer, BertModel
import torch
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

class ContextNeighborStorage:
    # def __init__(self, sentences, model):
    #     self.sentences = sentences
    #     self.model = model

    def __init__(self, sentences):
        self.sentences = sentences
        model = BertModel.from_pretrained('bert-base-uncased')
        self.model = model
        

    def process_sentences(self):
        inputs = tokenizer(self.sentences, return_tensors="pt" , padding=True,truncation=True)
        toks = inputs.input_ids.tolist()  
        outputs = model(**inputs)
        embs = outputs.last_hidden_state.tolist()
        result = [(toks,embs)]
        print(result)

        # last_hidden_states = outputs.last_hidden_state
        
        #result = self.model(**inputs)

        self.sentence_ids = []
        self.token_ids = []
        self.all_tokens = []
        all_embeddings = []

        for i,(toks, embs) in enumerate(tqdm(result)):
            print(i)

        # for i, (toks, embs) in enumerate(tqdm(result)):
        #     for j, (tok, emb) in enumerate(zip(toks, embs)):
        #         self.sentence_ids.append(i)
        #         self.token_ids.append(j)
        #         self.all_tokens.append(tok)
        #         all_embeddings.append(emb)
        # all_embeddings = np.stack(all_embeddings)
        # we normalize embeddings, so that euclidian distance is equivalent to cosine distance
        # self.normed_embeddings = (all_embeddings.T / (all_embeddings**2).sum(axis=1) ** 0.5).T

    def build_search_index(self):
        # this takes some time
        self.indexer = KDTree(self.normed_embeddings)

    def query(self, query_sent, query_word, k=10, filter_same_word=False):
        # toks, embs = self.model([query_sent])[0]
        inputs = tokenizer(query_sent, return_tensors="pt" , padding=True,truncation=True)
        toks = inputs.input_ids.tolist()[0]  
        outputs = model(**inputs)
        # last_hidden_states = outputs.last_hidden_state
        embs = outputs.last_hidden_state.tolist()

        found = False
        for tok, emb in zip(toks, embs):
            if tok == query_word:
                found = True
                break
        if not found:
            raise ValueError('The query word {} is not a single token in sentence {}'.format(query_word, toks))
        emb = emb / sum(emb**2)**0.5

        if filter_same_word:
            initial_k = max(k, 100)
        else:
            initial_k = k
        di, idx = self.indexer.query(emb.reshape(1, -1), k=initial_k)
        distances = []
        neighbors = []
        contexts = []
        for i, index in enumerate(idx.ravel()):
            token = self.all_tokens[index]
            if filter_same_word and (query_word in token or token in query_word):
                continue
            distances.append(di.ravel()[i])
            neighbors.append(token)
            contexts.append(self.sentences[self.sentence_ids[index]])
            if len(distances) == k:
                break
        return distances, neighbors, contexts

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
storage = ContextNeighborStorage()


TypeError: ignored

In [ ]:
distances, neighbors, contexts = storage.query(query_sent='It is a power bank.', query_word='bank', k=5)
for d, w, c in zip(distances, neighbors, contexts):
    print('{} {}  {}'.format(w, d, c.strip()))

ValueError: ignored